All models together to be run 50 times (once per subset/decluster)

**the modelling methodology**

“Each declustered subset (DCᵢ) was internally partitioned (80/20) to allow model validation on independent test data, ensuring that predictive performance and model stability were assessed under spatial independence. After internal validation, models were retrained on all samples in each DCᵢ to maximize predictive strength and produce the final maps used for ensemble averaging. The ensemble predictions were subsequently validated using the external holdout dataset (test₁) derived from the original data split.”

In [1]:
# No cross validation steps --All models together to be run 50 times (once per subset/decluster) 
#  =========================================================
# STEP — Run RF, OK, and RF+OK on all declustered subsets
# =========================================================

import os
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from pykrige.ok import OrdinaryKriging

# --- Paths ---
decluster_dir = "/Users/inesschwartz/Desktop/model/decluster_runs"
results_file = "/Users/inesschwartz/Desktop/model/results_all_models.csv"
output_dir = "/Users/inesschwartz/Desktop/model/predictions"
os.makedirs(output_dir, exist_ok=True)

# --- Hyperparameters (from tuning step) ---
rf_params = {
    'n_estimators': 1500,
    'min_samples_leaf': 3,
    'max_features': 0.5,
    'max_depth': None,
    'random_state': 42,
    'n_jobs': -1
}

# --- Fixed variogram parameters (from stability analysis) ---
variogram_params = {
    'model': 'gaussian',
    'variogram_parameters': {'nugget': 0.0, 'sill': 0.18, 'range': 14000}
}


# --- Helper for metrics ---
def compute_metrics(y_true, y_pred):
    return {
        'R2': r2_score(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }

# --- Results container ---
results = []

# --- Files ---
files = sorted(glob.glob(os.path.join(decluster_dir, "decluster_run_*.csv")))
print(f"Running RF, OK, and RF+OK for {len(files)} declustered subsets... ⏳")

# =========================================================
# MAIN LOOP
# =========================================================
for i, file in enumerate(files, start=1):
    df = pd.read_csv(file)
    target = 'log_soc_stock'

    # --- Train/test split ---
    train, test = train_test_split(df, test_size=0.2, random_state=42)
    X_train = train.drop(columns=[target, 'X_coord', 'Y_coord', 'site_info_id'], errors='ignore')
    y_train = train[target].values
    X_test = test.drop(columns=[target, 'X_coord', 'Y_coord', 'site_info_id'], errors='ignore')
    y_test = test[target].values

    coords_train = train[['X_coord', 'Y_coord']].values
    coords_test = test[['X_coord', 'Y_coord']].values

    # =====================================================
    # 1️⃣ Ordinary Kriging (OK)
    # =====================================================
    ok = OrdinaryKriging(
        x=coords_train[:, 0],
        y=coords_train[:, 1],
        z=y_train,
        variogram_model=variogram_params['model'],
        variogram_parameters=variogram_params['variogram_parameters'],
        enable_plotting=False,
        verbose=False
    )

    y_pred_ok, _ = ok.execute('points', coords_test[:, 0], coords_test[:, 1])
    metrics_ok = compute_metrics(y_test, y_pred_ok)
    results.append({'iteration': i, 'model': 'OK', **metrics_ok})

    # =====================================================
    # 2️⃣ Random Forest (RF)
    # =====================================================
    rf = RandomForestRegressor(**rf_params)
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    metrics_rf = compute_metrics(y_test, y_pred_rf)
    results.append({'iteration': i, 'model': 'RF', **metrics_rf})

    # =====================================================
    # 3️⃣ Hybrid RF + OK (residual kriging)
    # =====================================================
    residuals = y_train - rf.predict(X_train)
    ok_resid = OrdinaryKriging(
        x=coords_train[:, 0],
        y=coords_train[:, 1],
        z=residuals,
        variogram_model=variogram_params['model'],
        variogram_parameters=variogram_params['variogram_parameters'],
        enable_plotting=False,
        verbose=False
    )

    residual_pred, _ = ok_resid.execute('points', coords_test[:, 0], coords_test[:, 1])
    y_pred_rfok = y_pred_rf + residual_pred

    metrics_rfok = compute_metrics(y_test, y_pred_rfok)
    results.append({'iteration': i, 'model': 'RF+OK', **metrics_rfok})

    print(f"✅ Iteration {i:02d} complete | RF R²={metrics_rf['R2']:.3f}, OK R²={metrics_ok['R2']:.3f}, RF+OK R²={metrics_rfok['R2']:.3f}")

    # --- Save iteration predictions (optional) ---
    out = test[['X_coord', 'Y_coord', target]].copy()
    out['pred_RF'] = y_pred_rf
    out['pred_OK'] = y_pred_ok
    out['pred_RF_OK'] = y_pred_rfok
    out.to_csv(os.path.join(output_dir, f"predictions_iter_{i:03d}.csv"), index=False)

# =====================================================
# Save all metrics
# =====================================================
results_df = pd.DataFrame(results)
results_df.to_csv(results_file, index=False)
print(f"\n📁 All model results saved to: {results_file}")


Running RF, OK, and RF+OK for 50 declustered subsets... ⏳
✅ Iteration 01 complete | RF R²=0.332, OK R²=-0.089, RF+OK R²=0.275
✅ Iteration 02 complete | RF R²=0.319, OK R²=-0.086, RF+OK R²=0.232
✅ Iteration 03 complete | RF R²=0.329, OK R²=-0.039, RF+OK R²=0.269
✅ Iteration 04 complete | RF R²=0.316, OK R²=-0.045, RF+OK R²=0.251
✅ Iteration 05 complete | RF R²=0.335, OK R²=-0.091, RF+OK R²=0.278
✅ Iteration 06 complete | RF R²=0.349, OK R²=-0.102, RF+OK R²=0.260
✅ Iteration 07 complete | RF R²=0.321, OK R²=-0.055, RF+OK R²=0.286
✅ Iteration 08 complete | RF R²=0.324, OK R²=-0.057, RF+OK R²=0.282
✅ Iteration 09 complete | RF R²=0.326, OK R²=-0.043, RF+OK R²=0.262
✅ Iteration 10 complete | RF R²=0.328, OK R²=-0.084, RF+OK R²=0.273
✅ Iteration 11 complete | RF R²=0.328, OK R²=-0.084, RF+OK R²=0.273
✅ Iteration 12 complete | RF R²=0.332, OK R²=-0.088, RF+OK R²=0.276
✅ Iteration 13 complete | RF R²=0.323, OK R²=-0.042, RF+OK R²=0.260
✅ Iteration 14 complete | RF R²=0.315, OK R²=0.001, RF+OK 

In [4]:
# # (Calibration Diagnostics Only) didn't handle ok data well...
# # =========================================================
# # STEP — Decluster modeling loop (Calibration Diagnostics Only)
# # =========================================================

# import os
# import glob
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
# from sklearn.model_selection import KFold
# from pykrige.ok import OrdinaryKriging
# import warnings
# warnings.filterwarnings("ignore")

# # =========================================================
# # PATHS AND PARAMETERS
# # =========================================================
# decluster_dir = "/Users/inesschwartz/Desktop/model/decluster_runs"
# output_dir = "/Users/inesschwartz/Desktop/model/results_calibration"
# os.makedirs(output_dir, exist_ok=True)

# # --- RF parameters (from tuning) ---
# rf_params = {
#     'n_estimators': 1500,
#     'min_samples_leaf': 3,
#     'max_features': 0.5,
#     'max_depth': None,
#     'random_state': 42,
#     'n_jobs': 4
# }

# # Variogram parameters
# variogram_model = 'exponential'
# variogram_params = {'nugget': 0.0022, 'sill': 0.22, 'range': 15000}

# # =========================================================
# # HELPER FUNCTIONS
# # =========================================================
# def compute_metrics(y_true, y_pred):
#     """Compute standard regression metrics."""
#     return {
#         'R2': r2_score(y_true, y_pred),
#         'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
#         'MAE': mean_absolute_error(y_true, y_pred)
#     }

# def cross_validate_rf(X, y, k=5):
#     """Run k-fold cross-validation for Random Forest."""
#     kf = KFold(n_splits=k, shuffle=True, random_state=42)
#     metrics = []

#     for train_idx, test_idx in kf.split(X):
#         X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_test = y[train_idx], y[test_idx]

#         rf = RandomForestRegressor(**rf_params)
#         rf.fit(X_train, y_train)
#         y_pred = rf.predict(X_test)
#         metrics.append(compute_metrics(y_test, y_pred))

#     avg = pd.DataFrame(metrics).mean().to_dict()
#     return avg

# def run_ok(coords, values, variogram_params, model='exponential'):
#     """Run Ordinary Kriging with fixed variogram parameters."""
#     ok = OrdinaryKriging(
#         x=coords[:, 0],
#         y=coords[:, 1],
#         z=values,
#         variogram_model=model,
#         variogram_parameters={
#             'sill': variogram_params['sill'],
#             'range': variogram_params['range'],
#             'nugget': variogram_params['nugget']
#         },
#         enable_plotting=False,
#         verbose=False
#     )
#     return ok

# def run_rf_ok(X, y, coords, variogram_params):
#     """Run hybrid RF + OK (residual kriging)."""
#     rf = RandomForestRegressor(**rf_params)
#     rf.fit(X, y)
#     y_pred_rf = rf.predict(X)
#     residuals = y - y_pred_rf

#     ok = run_ok(coords, residuals, variogram_params)
#     residual_pred, _ = ok.execute('points', coords[:, 0], coords[:, 1])
#     y_pred_hybrid = y_pred_rf + residual_pred.data
#     return compute_metrics(y, y_pred_hybrid)

# # =========================================================
# # MAIN CALIBRATION LOOP
# # =========================================================
# files = sorted(glob.glob(os.path.join(decluster_dir, "decluster_run_*.csv")))
# n_calibration = 10  # limit to first 10 DCs

# print(f"🔁 Running calibration for first {n_calibration} declustered subsets...")

# calibration_results = []

# for i, file in enumerate(files[:n_calibration], start=1):
#     df = pd.read_csv(file)
#     print(f"\n📂 Processing decluster subset {i:03d} | n = {len(df)}")

#     target = 'log_soc_stock'
#     X = df.drop(columns=[target, 'X_coord', 'Y_coord', 'site_info_id'], errors='ignore')
#     y = df[target].values
#     coords = df[['X_coord', 'Y_coord']].values

#     print("⚙️ Running internal 5-fold CV and validation...")

#     # --- RF Cross-validation ---
#     rf_cv_metrics = cross_validate_rf(X, y, k=5)

#     # --- OK validation (LOO-style) --- CAN SKIP THIS
# #     ok = run_ok(coords, y, variogram_params)
#     y_pred_ok, _ = ok.execute('points', coords[:, 0], coords[:, 1])
#     ok_metrics = compute_metrics(y, y_pred_ok)

#     # --- RF+OK residual kriging ---
#     hybrid_metrics = run_rf_ok(X, y, coords, variogram_params)

#     # --- Store results ---
#     calibration_results.append({'subset': i, 'model': 'RF_CV', **rf_cv_metrics})
#     calibration_results.append({'subset': i, 'model': 'OK', **ok_metrics})
#     calibration_results.append({'subset': i, 'model': 'RF+OK', **hybrid_metrics})

#     print(f"✅ DC{i:03d} | RF R²={rf_cv_metrics['R2']:.3f} | OK R²={ok_metrics['R2']:.3f} | Hybrid R²={hybrid_metrics['R2']:.3f}")

# # =========================================================
# # SAVE RESULTS
# # =========================================================
# out_csv = os.path.join(output_dir, "calibration_results.csv")
# pd.DataFrame(calibration_results).to_csv(out_csv, index=False)

# print("\n✅ Calibration diagnostics complete.")
# print(f"📁 Results saved to: {out_csv}")


🔁 Running calibration for first 10 declustered subsets...

📂 Processing decluster subset 001 | n = 629
⚙️ Running internal 5-fold CV and validation...
✅ DC001 | RF R²=0.333 | OK R²=1.000 | Hybrid R²=1.000

📂 Processing decluster subset 002 | n = 629
⚙️ Running internal 5-fold CV and validation...
✅ DC002 | RF R²=0.304 | OK R²=1.000 | Hybrid R²=1.000

📂 Processing decluster subset 003 | n = 629
⚙️ Running internal 5-fold CV and validation...
✅ DC003 | RF R²=0.321 | OK R²=1.000 | Hybrid R²=1.000

📂 Processing decluster subset 004 | n = 629
⚙️ Running internal 5-fold CV and validation...
✅ DC004 | RF R²=0.318 | OK R²=1.000 | Hybrid R²=1.000

📂 Processing decluster subset 005 | n = 629
⚙️ Running internal 5-fold CV and validation...
✅ DC005 | RF R²=0.325 | OK R²=1.000 | Hybrid R²=1.000

📂 Processing decluster subset 006 | n = 629
⚙️ Running internal 5-fold CV and validation...
✅ DC006 | RF R²=0.317 | OK R²=1.000 | Hybrid R²=1.000

📂 Processing decluster subset 007 | n = 629
⚙️ Running inte

OLD: 
Calibration results summary (DC₁–₁₀):
Random Forest shows consistent moderate predictive ability (R² ≈ 0.32, RMSE ≈ 0.44).
However, Ordinary Kriging (OK) and hybrid RF+OK show perfect interpolation (R²=1.0), indicating they were evaluated on the same training data rather than withheld samples.
The next step is to implement Leave-One-Out (LOO) kriging validation to obtain unbiased performance estimates for OK and RF+OK before proceeding to ensemble modeling.

In [1]:
# =========================================================
# STEP — Decluster modeling loop with Spatial K-Fold & LOO
# =========================================================

import os
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold
from pykrige.ok import OrdinaryKriging
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# =========================================================
# PATHS AND PARAMETERS
# =========================================================
decluster_dir = "/Users/inesschwartz/Desktop/model/decluster_runs"
output_dir = "/Users/inesschwartz/Desktop/model/results_final"
os.makedirs(output_dir, exist_ok=True)

# Random Forest tuned parameters
rf_params = {
    'n_estimators': 1500,
    'min_samples_leaf': 3,
    'max_features': 0.5,
    'max_depth': None,
    'random_state': 42,
    'n_jobs': 4
}

# Variogram parameters (from calibration)
variogram_model = 'exponential'
variogram_params = {'nugget': 0.0022, 'sill': 0.22, 'range': 15000}

# =========================================================
# HELPER FUNCTIONS
# =========================================================
def compute_metrics(y_true, y_pred):
    """Compute standard regression metrics."""
    return {
        'R2': r2_score(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }


def spatial_kfold(coords, n_splits=5):
    """
    Custom spatial K-Fold generator.
    Splits data into roughly spatially distinct clusters
    by sorting coordinates and splitting sequentially.
    """
    # Sort by X + Y coordinate to ensure spatial grouping
    idx = np.argsort(coords[:, 0] + coords[:, 1])
    fold_sizes = np.full(n_splits, len(coords) // n_splits, dtype=int)
    fold_sizes[:len(coords) % n_splits] += 1
    current = 0
    folds = []
    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        test_idx = idx[start:stop]
        train_idx = np.setdiff1d(np.arange(len(coords)), test_idx)
        folds.append((train_idx, test_idx))
        current = stop
    return folds


def cross_validate_rf_spatial(X, y, coords, n_splits=5):
    """Spatial K-Fold cross-validation for Random Forest."""
    folds = spatial_kfold(coords, n_splits)
    metrics = []
    for train_idx, test_idx in folds:
        rf = RandomForestRegressor(**rf_params)
        rf.fit(X.iloc[train_idx], y[train_idx])
        y_pred = rf.predict(X.iloc[test_idx])
        metrics.append(compute_metrics(y[test_idx], y_pred))
    return pd.DataFrame(metrics).mean().to_dict()


def loo_validate_ok(coords, values, variogram_params, model='exponential'):
    """Leave-One-Out Cross-Validation for Ordinary Kriging."""
    preds = np.zeros(len(values))
    for i in range(len(values)):
        mask = np.ones(len(values), dtype=bool)
        mask[i] = False
        ok = OrdinaryKriging(
            x=coords[mask, 0],
            y=coords[mask, 1],
            z=values[mask],
            variogram_model=model,
            variogram_parameters={
                'sill': variogram_params['sill'],
                'range': variogram_params['range'],
                'nugget': variogram_params['nugget']
            },
            enable_plotting=False,
            verbose=False
        )
        zhat, _ = ok.execute('points',
                             np.array([coords[i, 0]]),
                             np.array([coords[i, 1]]))
        preds[i] = zhat.data[0]
    return compute_metrics(values, preds)


def loo_validate_rf_ok(X, y, coords, variogram_params):
    """LOO validation for hybrid RF+OK (residual kriging)."""
    preds = np.zeros(len(y))
    for i in range(len(y)):
        mask = np.ones(len(y), dtype=bool)
        mask[i] = False
        rf = RandomForestRegressor(**rf_params)
        rf.fit(X.iloc[mask], y[mask])
        y_pred_rf = rf.predict(X.iloc[mask])
        residuals = y[mask] - y_pred_rf
        ok = OrdinaryKriging(
            x=coords[mask, 0],
            y=coords[mask, 1],
            z=residuals,
            variogram_model='exponential',
            variogram_parameters={
                'sill': variogram_params['sill'],
                'range': variogram_params['range'],
                'nugget': variogram_params['nugget']
            },
            enable_plotting=False,
            verbose=False
        )
        zhat, _ = ok.execute('points',
                             np.array([coords[i, 0]]),
                             np.array([coords[i, 1]]))
        yhat_hybrid = rf.predict(X.iloc[[i]])[0] + zhat.data[0]
        preds[i] = yhat_hybrid
    return compute_metrics(y, preds)


# =========================================================
# MAIN CALIBRATION LOOP
# =========================================================
files = sorted(glob.glob(os.path.join(decluster_dir, "decluster_run_*.csv")))
calibration_results = []

print(f"🔁 Running RF (Spatial K-Fold), OK (LOO), and RF+OK (LOO) for {len(files[:10])} decluster subsets...")

for i, file in enumerate(tqdm(files[:10], desc="Calibration progress", ncols=100), start=1):
    df = pd.read_csv(file)
    target = 'log_soc_stock'
    X = df.drop(columns=[target, 'X_coord', 'Y_coord', 'site_info_id'], errors='ignore')
    y = df[target].values
    coords = df[['X_coord', 'Y_coord']].values

    rf_spatial = cross_validate_rf_spatial(X, y, coords, n_splits=5)
    ok_loo = loo_validate_ok(coords, y, variogram_params, model=variogram_model)
    hybrid_loo = loo_validate_rf_ok(X, y, coords, variogram_params)

    calibration_results.append({'subset': i, 'model': 'RF_SpatialKFold', **rf_spatial})
    calibration_results.append({'subset': i, 'model': 'OK_LOO', **ok_loo})
    calibration_results.append({'subset': i, 'model': 'RF+OK_LOO', **hybrid_loo})

# --- Save calibration results ---
calib_path = os.path.join(output_dir, "calibration_results_spatial.csv")
pd.DataFrame(calibration_results).to_csv(calib_path, index=False)

print(f"\n📁 Calibration results saved to: {calib_path}")
print("✅ Spatial K-Fold + LOO calibration complete.")


🔁 Running RF (Spatial K-Fold), OK (LOO), and RF+OK (LOO) for 10 decluster subsets...


Calibration progress: 100%|█████████████████████████████████████| 10/10 [3:18:05<00:00, 1188.54s/it]


📁 Calibration results saved to: /Users/inesschwartz/Desktop/model/results_final/calibration_results_spatial.csv
✅ Spatial K-Fold + LOO calibration complete.


In [1]:
# ## ok rf-ok trial

#  # =========================================================
# # STEP — Calibration test for OK and RF+OK (5 declusters)
# # =========================================================

# import os
# import glob
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
# from pykrige.ok import OrdinaryKriging
# from tqdm import tqdm
# import warnings
# warnings.filterwarnings("ignore")

# # =========================================================
# # PATHS AND PARAMETERS
# # =========================================================
# decluster_dir = "/Users/inesschwartz/Desktop/model/decluster_runs"
# output_dir = "/Users/inesschwartz/Desktop/model/results_calibration_ok"
# os.makedirs(output_dir, exist_ok=True)

# # Variogram parameters (from your calibration step)
# variogram_model = 'exponential'
# variogram_params = {'nugget': 0.0022, 'sill': 0.22, 'range': 15000}

# # Random Forest tuned parameters
# rf_params = {
#     'n_estimators': 1500,
#     'min_samples_leaf': 3,
#     'max_features': 0.5,
#     'max_depth': None,
#     'random_state': 42,
#     'n_jobs': -1
# }

# # =========================================================
# # HELPER FUNCTIONS
# # =========================================================
# def compute_metrics(y_true, y_pred):
#     """Compute regression metrics."""
#     return {
#         'R2': r2_score(y_true, y_pred),
#         'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
#         'MAE': mean_absolute_error(y_true, y_pred)
#     }


# def loo_validate_ok(coords, values, variogram_params, model='exponential'):
#     """Leave-One-Out Cross-Validation for Ordinary Kriging."""
#     preds = np.zeros(len(values))
#     for i in range(len(values)):
#         mask = np.ones(len(values), dtype=bool)
#         mask[i] = False

#         ok = OrdinaryKriging(
#             x=coords[mask, 0],
#             y=coords[mask, 1],
#             z=values[mask],
#             variogram_model=model,
#             variogram_parameters=[
#                 variogram_params['sill'],
#                 variogram_params['range'],
#                 variogram_params['nugget']
#             ],
#             enable_plotting=False,
#             verbose=False
#         )

#         zhat, _ = ok.execute('points',
#                              np.array([coords[i, 0]]),
#                              np.array([coords[i, 1]]))
#         preds[i] = zhat.data[0]

#     return compute_metrics(values, preds)


# def loo_validate_rf_ok(X, y, coords, variogram_params):
#     """Leave-One-Out validation for hybrid RF+OK (residual kriging)."""
#     preds = np.zeros(len(y))
#     for i in range(len(y)):
#         mask = np.ones(len(y), dtype=bool)
#         mask[i] = False

#         rf = RandomForestRegressor(**rf_params)
#         rf.fit(X.iloc[mask], y[mask])
#         y_pred_rf = rf.predict(X.iloc[mask])
#         residuals = y[mask] - y_pred_rf

#         ok = OrdinaryKriging(
#             x=coords[mask, 0],
#             y=coords[mask, 1],
#             z=residuals,
#             variogram_model='exponential',
#             variogram_parameters=[
#                 variogram_params['sill'],
#                 variogram_params['range'],
#                 variogram_params['nugget']
#             ],
#             enable_plotting=False,
#             verbose=False
#         )

#         zhat, _ = ok.execute('points',
#                              np.array([coords[i, 0]]),
#                              np.array([coords[i, 1]]))
#         yhat_hybrid = rf.predict(X.iloc[[i]])[0] + zhat.data[0]
#         preds[i] = yhat_hybrid

#     return compute_metrics(y, preds)

# # =========================================================
# # MAIN CALIBRATION LOOP
# # =========================================================
# files = sorted(glob.glob(os.path.join(decluster_dir, "decluster_run_*.csv")))[:5]
# results = []

# print(f"🔁 Running OK and RF+OK (LOO) for {len(files)} declustered subsets...")

# for i, file in enumerate(tqdm(files, desc="Calibration progress", ncols=100), start=1):
#     df = pd.read_csv(file)
#     target = 'log_soc_stock'
#     X = df.drop(columns=[target, 'X_coord', 'Y_coord', 'site_info_id'], errors='ignore')
#     y = df[target].values
#     coords = df[['X_coord', 'Y_coord']].values

#     ok_metrics = loo_validate_ok(coords, y, variogram_params, model=variogram_model)
#     hybrid_metrics = loo_validate_rf_ok(X, y, coords, variogram_params)

#     results.append({'subset': i, 'model': 'OK_LOO', **ok_metrics})
#     results.append({'subset': i, 'model': 'RF+OK_LOO', **hybrid_metrics})

# # =========================================================
# # SAVE & DISPLAY RESULTS
# # =========================================================
# results_df = pd.DataFrame(results)
# out_path = os.path.join(output_dir, "ok_rfok_calibration_results.csv")
# results_df.to_csv(out_path, index=False)

# print(f"\n📁 Calibration results saved to: {out_path}")
# print("✅ Calibration complete.\n")

# print("Summary (mean across 5 subsets):")
# print(results_df.groupby("model")[["R2", "RMSE", "MAE"]].mean().round(3))


🔁 Running OK and RF+OK (LOO) for 5 declustered subsets...


Calibration progress: 100%|███████████████████████████████████████| 5/5 [1:55:39<00:00, 1387.82s/it]


📁 Calibration results saved to: /Users/inesschwartz/Desktop/model/results_calibration_ok/ok_rfok_calibration_results.csv
✅ Calibration complete.

Summary (mean across 5 subsets):
              R2   RMSE    MAE
model                         
OK_LOO     0.085  0.514  0.392
RF+OK_LOO  0.332  0.439  0.315


In [ ]:
## full dc runs

# =========================================================
# STEP — Decluster modeling loop with Spatial K-Fold & LOO
# =========================================================

import os
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold
from pykrige.ok import OrdinaryKriging
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# =========================================================
# PATHS AND PARAMETERS
# =========================================================
decluster_dir = "/Users/inesschwartz/Desktop/model/decluster_runs"
output_dir = "/Users/inesschwartz/Desktop/model/results_final"
os.makedirs(output_dir, exist_ok=True)

# Random Forest tuned parameters
rf_params = {
    'n_estimators': 1500,
    'min_samples_leaf': 3,
    'max_features': 0.5,
    'max_depth': None,
    'random_state': 42,
    'n_jobs': -1
}

# Variogram parameters (from your calibration step)
variogram_model = 'exponential'
variogram_params = {'nugget': 0.0022, 'sill': 0.22, 'range': 15000}

# =========================================================
# HELPER FUNCTIONS
# =========================================================
def compute_metrics(y_true, y_pred):
    """Compute standard regression metrics."""
    return {
        'R2': r2_score(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }


def spatial_kfold(coords, n_splits=5):
    """
    Custom spatial K-Fold generator.
    Splits data into roughly spatially distinct clusters
    by sorting coordinates and splitting sequentially.
    """
    # Sort by X + Y coordinate to ensure spatial grouping
    idx = np.argsort(coords[:, 0] + coords[:, 1])
    fold_sizes = np.full(n_splits, len(coords) // n_splits, dtype=int)
    fold_sizes[:len(coords) % n_splits] += 1
    current = 0
    folds = []
    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        test_idx = idx[start:stop]
        train_idx = np.setdiff1d(np.arange(len(coords)), test_idx)
        folds.append((train_idx, test_idx))
        current = stop
    return folds


def cross_validate_rf_spatial(X, y, coords, n_splits=5):
    """Spatial K-Fold cross-validation for Random Forest."""
    folds = spatial_kfold(coords, n_splits)
    metrics = []
    for train_idx, test_idx in folds:
        rf = RandomForestRegressor(**rf_params)
        rf.fit(X.iloc[train_idx], y[train_idx])
        y_pred = rf.predict(X.iloc[test_idx])
        metrics.append(compute_metrics(y[test_idx], y_pred))
    return pd.DataFrame(metrics).mean().to_dict()


def loo_validate_ok(coords, values, variogram_params, model='exponential'):
    """Leave-One-Out Cross-Validation for Ordinary Kriging."""
    preds = np.zeros(len(values))
    for i in range(len(values)):
        mask = np.ones(len(values), dtype=bool)
        mask[i] = False

        ok = OrdinaryKriging(
            x=coords[mask, 0],
            y=coords[mask, 1],
            z=values[mask],
            variogram_model=model,
            variogram_parameters=[
                variogram_params['sill'],
                variogram_params['range'],
                variogram_params['nugget']
            ],
            enable_plotting=False,
            verbose=False
        )

        zhat, _ = ok.execute('points',
                             np.array([coords[i, 0]]),
                             np.array([coords[i, 1]]))
        preds[i] = zhat.data[0]

    return compute_metrics(values, preds)


def loo_validate_rf_ok(X, y, coords, variogram_params):
    """Leave-One-Out validation for hybrid RF+OK (residual kriging)."""
    preds = np.zeros(len(y))
    for i in range(len(y)):
        mask = np.ones(len(y), dtype=bool)
        mask[i] = False

        rf = RandomForestRegressor(**rf_params)
        rf.fit(X.iloc[mask], y[mask])
        y_pred_rf = rf.predict(X.iloc[mask])
        residuals = y[mask] - y_pred_rf

        ok = OrdinaryKriging(
            x=coords[mask, 0],
            y=coords[mask, 1],
            z=residuals,
            variogram_model='exponential',
            variogram_parameters=[
                variogram_params['sill'],
                variogram_params['range'],
                variogram_params['nugget']
            ],
            enable_plotting=False,
            verbose=False
        )

        zhat, _ = ok.execute('points',
                             np.array([coords[i, 0]]),
                             np.array([coords[i, 1]]))
        yhat_hybrid = rf.predict(X.iloc[[i]])[0] + zhat.data[0]
        preds[i] = yhat_hybrid

    return compute_metrics(y, preds)

# =========================================================
# MAIN CALIBRATION LOOP # can take out now that I ran and liked the results?
# =========================================================
files = sorted(glob.glob(os.path.join(decluster_dir, "decluster_run_*.csv")))[:5]
results = []

print(f"🔁 Running OK and RF+OK (LOO) for {len(files)} declustered subsets...")

for i, file in enumerate(tqdm(files, desc="Calibration progress", ncols=100), start=1):
    df = pd.read_csv(file)
    target = 'log_soc_stock'
    X = df.drop(columns=[target, 'X_coord', 'Y_coord', 'site_info_id'], errors='ignore')
    y = df[target].values
    coords = df[['X_coord', 'Y_coord']].values

    ok_metrics = loo_validate_ok(coords, y, variogram_params, model=variogram_model)
    hybrid_metrics = loo_validate_rf_ok(X, y, coords, variogram_params)

    results.append({'subset': i, 'model': 'OK_LOO', **ok_metrics})
    results.append({'subset': i, 'model': 'RF+OK_LOO', **hybrid_metrics})

# =========================================================
# SAVE & DISPLAY RESULTS
# =========================================================
results_df = pd.DataFrame(results)
out_path = os.path.join(output_dir, "ok_rfok_calibration_results.csv")
results_df.to_csv(out_path, index=False)

print(f"\n📁 Calibration results saved to: {out_path}")
print("✅ Calibration complete.\n")

print("Summary (mean across 5 subsets):")
print(results_df.groupby("model")[["R2", "RMSE", "MAE"]].mean().round(3))


## Final map prediction all 3 algorithms over declusters 1-50 (in batches to handle memory)

In [12]:
# #1st try, working but results not great and no performance metrics 
# # =========================================================
# # Usage examples:
# #   python SOC_ensemble_main.py 1 10
# #   python SOC_ensemble_main.py 11 20
# #   python SOC_ensemble_main.py 21 30
# # =========================================================

# import os, sys, glob, gc
# import pandas as pd
# import numpy as np
# import rasterio
# from rasterio.transform import rowcol
# from sklearn.ensemble import RandomForestRegressor
# from pykrige.ok import OrdinaryKriging

# # =========================================================
# # USER PARAMETERS
# # =========================================================
# decluster_dir = "/Users/inesschwartz/Desktop/model/decluster_runs_aligned"
# pred_grid_csv = "/Users/inesschwartz/Desktop/model/covariates_stack_1km_utm33s_fixed.csv"
# ref_raster = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/slope_height.tif"
# output_dir = "/Users/inesschwartz/Desktop/model/ensemble_preds"
# os.makedirs(output_dir, exist_ok=True)

# target = "log_soc_stock"

# rf_params = {
#     "n_estimators": 500,
#     "min_samples_leaf": 3,
#     "max_features": 0.5,
#     "max_depth": 20,
#     "random_state": 42,
#     "n_jobs": 4  # limit to avoid overload
# }

# variogram_params = {
#     "model": "gaussian",
#     "variogram_parameters": {"nugget": 0.0, "sill": 0.18, "range": 14000}
# }

# features = [
#     "MRRTF", "MRVBF", "annual_precip", "grazing_1950", "cropland_1950",
#     "precip_wettest_month", "relief_TRI", "standardized_height",
#     "temp_annual_range", "terrain_surf_convexity", "terrain_surf_texture",
#     "tmax_mean", "valley_depth", "faosoil_id", "slope_height"
# ]

# # =========================================================
# # OPTIONAL: command line batch arguments
# # =========================================================
# if len(sys.argv) >= 3:
#     start_idx = int(sys.argv[1])
#     end_idx = int(sys.argv[2])
# else:
#     start_idx, end_idx = 1, 10  # default batch (1–10)

# print(f"\n🚀 Running SOC ensemble for declusters {start_idx}–{end_idx}...")

# # =========================================================
# # LOAD PREDICTION GRID (same for all declusters)
# # =========================================================
# df_pred = pd.read_csv(pred_grid_csv)
# X_pred = df_pred[features].astype("float32").copy()
# coords_pred = df_pred[["X_coord", "Y_coord"]].astype("float32").values

# # Reference raster geometry
# with rasterio.open(ref_raster) as ref:
#     profile = ref.profile
#     transform = ref.transform
#     width, height = ref.width, ref.height
#     crs = ref.crs

# # Precompute row/col mapping (outside loop)
# rows, cols = rowcol(transform, df_pred["X_coord"], df_pred["Y_coord"])
# rows, cols = np.array(rows), np.array(cols)
# valid_mask = (
#     (rows >= 0) & (rows < height) &
#     (cols >= 0) & (cols < width)
# )

# # =========================================================
# # MEMORY-SAFE KRIGING FUNCTION
# # =========================================================
# def krige_in_blocks(ok_model, coords_pred, block_size=10000):
#     preds = []
#     for start in range(0, len(coords_pred), block_size):
#         end = start + block_size
#         subset = coords_pred[start:end]
#         pred_block, _ = ok_model.execute("points", subset[:, 0], subset[:, 1])
#         preds.append(pred_block)
#     return np.concatenate(preds)

# # =========================================================
# # LOOP THROUGH DECLUSTERS (BATCH RANGE)
# # =========================================================
# decluster_files = sorted(glob.glob(os.path.join(decluster_dir, "decluster_run_*.csv")))
# subset_files = decluster_files[start_idx-1:end_idx]

# print(f"📂 Running 3 models × {len(subset_files)} declusters in this batch...")

# for i, file in enumerate(subset_files, start=start_idx):
#     print(f"\n--- Decluster {i:02d} ---")
#     df_train = pd.read_csv(file)
#     X_train = df_train[features].astype("float32").copy()
#     y_train = df_train[target].astype("float32").copy()
#     coords_train = df_train[["X_coord", "Y_coord"]].astype("float32").values

#     # -----------------------------------------------------
#     # 1️⃣ RANDOM FOREST (RF)
#     # -----------------------------------------------------
#     rf = RandomForestRegressor(**rf_params)
#     rf.fit(X_train, y_train)
#     pred_rf = rf.predict(X_pred)

#     # -----------------------------------------------------
#     # 2️⃣ ORDINARY KRIGING (OK)
#     # -----------------------------------------------------
#     ok = OrdinaryKriging(
#         x=coords_train[:, 0],
#         y=coords_train[:, 1],
#         z=y_train,
#         variogram_model=variogram_params["model"],
#         variogram_parameters=variogram_params["variogram_parameters"],
#         enable_plotting=False,
#         verbose=False
#     )
#     pred_ok = krige_in_blocks(ok, coords_pred)

#     # -----------------------------------------------------
#     # 3️⃣ HYBRID RF + OK (Residual Kriging)
#     # -----------------------------------------------------
#     residuals = y_train - rf.predict(X_train)
#     ok_resid = OrdinaryKriging(
#         x=coords_train[:, 0],
#         y=coords_train[:, 1],
#         z=residuals,
#         variogram_model=variogram_params["model"],
#         variogram_parameters=variogram_params["variogram_parameters"],
#         enable_plotting=False,
#         verbose=False
#     )
#     pred_resid = krige_in_blocks(ok_resid, coords_pred)
#     pred_rfok = pred_rf + pred_resid

#     # -----------------------------------------------------
#     # BACK-TRANSFORM (log → SOC) AND SAVE RASTERS
#     # -----------------------------------------------------
#     model_preds = {
#         "RF": np.expm1(pred_rf),
#         "OK": np.expm1(pred_ok),
#         "RF_OK": np.expm1(pred_rfok)
#     }

#     for model_name, values in model_preds.items():
#         arr = np.full((height, width), np.nan, dtype="float32")
#         arr[rows[valid_mask], cols[valid_mask]] = values[valid_mask]
#         out_tif = os.path.join(output_dir, f"{model_name}_decluster_{i:03d}.tif")

#         profile.update(dtype="float32", count=1, compress="lzw", nodata=np.nan)
#         with rasterio.open(out_tif, "w", **profile) as dst:
#             dst.write(arr, 1)

#     print(f"✅ Decluster {i:02d} complete → RF, OK, RF+OK rasters saved.")

#     # Clean up memory before next iteration
#     del rf, ok, ok_resid, X_train, y_train, coords_train, pred_rf, pred_ok, pred_resid, pred_rfok, model_preds, arr
#     gc.collect()

# print(f"\n🎯 Batch {start_idx}–{end_idx} complete. Rasters saved to:\n{output_dir}")


## declusters--> models --> perfomance metrics and tifs --> aggregated ensemble for final

In [1]:
#1 cross validation metrics 

# =========================================================
# SOC_spatialCV_metrics.py
# =========================================================
# Runs 5-fold spatial CV (10 km GroupKFold) for RF, OK, RF+OK.
# Saves per-fold and per-decluster metrics.
# =========================================================

import os, sys, glob, gc
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pykrige.ok import OrdinaryKriging

# ---------- User parameters ----------
decluster_dir = "/Users/inesschwartz/Desktop/model/decluster_runs_aligned"
metrics_dir   = "/Users/inesschwartz/Desktop/model/results_spatialCV"
os.makedirs(metrics_dir, exist_ok=True)

target = "log_soc_stock"
features = [
    "MRRTF", "MRVBF", "annual_precip", "grazing_1950", "cropland_1950",
    "precip_wettest_month", "relief_TRI", "standardized_height",
    "temp_annual_range", "terrain_surf_convexity", "terrain_surf_texture",
    "tmax_mean", "valley_depth", "faosoil_id", "slope_height"
]

rf_params = dict(
    n_estimators=500, min_samples_leaf=3, max_features=0.5,
    max_depth=20, random_state=42, n_jobs=4
)
vmodel = "gaussian"
vparams = {"nugget": 0.0, "sill": 0.18, "range": 14000}

# ---------- Batch arguments ----------
if len(sys.argv) >= 3:
    start_idx, end_idx = int(sys.argv[1]), int(sys.argv[2])
else:
    start_idx, end_idx = 1, 10

print(f"\n🚀 Running spatial CV for declusters {start_idx}–{end_idx}")

# ---------- Helpers ----------
def block_groups(df, block_m=10000):
    gx = (df["X_coord"] // block_m).astype(int)
    gy = (df["Y_coord"] // block_m).astype(int)
    return (gx.astype(str) + "_" + gy.astype(str)).values

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def krige_predict(xy_train, z_train, xy_test):
    ok = OrdinaryKriging(
        x=xy_train[:,0], y=xy_train[:,1], z=z_train,
        variogram_model=vmodel, variogram_parameters=vparams,
        enable_plotting=False, verbose=False
    )
    pred, _ = ok.execute("points", xy_test[:,0], xy_test[:,1])
    return np.asarray(pred)

# ---------- Main loop ----------
decluster_files = sorted(glob.glob(os.path.join(decluster_dir, "decluster_run_*.csv")))
subset = decluster_files[start_idx-1:end_idx]
all_rows = []

for path in subset:
    name = os.path.basename(path)
    it = int(name.split("_")[-1].split(".")[0])
    print(f"\n🧩 Decluster {it:03d}")

    df = pd.read_csv(path)
    df = df.dropna(subset=[target]+features+["X_coord","Y_coord"])

    X = df[features].astype("float32").values
    y = df[target].astype("float32").values
    XY = df[["X_coord","Y_coord"]].astype("float32").values
    groups = block_groups(df, block_m=10000)
    cv = GroupKFold(n_splits=5)

    fold = 0
    for train_idx, test_idx in cv.split(X, y, groups=groups):
        fold += 1
        Xtr, Xte = X[train_idx], X[test_idx]
        ytr, yte = y[train_idx], y[test_idx]
        XYtr, XYte = XY[train_idx], XY[test_idx]

        # RF
        rf = RandomForestRegressor(**rf_params)
        rf.fit(Xtr, ytr)
        rf_te = rf.predict(Xte)

        # OK
        ok_te = krige_predict(XYtr, ytr, XYte)

        # RF+OK
        resid_tr = ytr - rf.predict(Xtr)
        rk_te = rf_te + krige_predict(XYtr, resid_tr, XYte)

        for model_name, preds in (("RF", rf_te), ("OK", ok_te), ("RF_OK", rk_te)):
            all_rows.append({
                "decluster": it, "fold": fold, "model": model_name,
                "R2": r2_score(yte, preds),
                "RMSE": rmse(yte, preds),
                "MAE": mean_absolute_error(yte, preds)
            })
        gc.collect()

# ---------- Save metrics ----------
metrics_df = pd.DataFrame(all_rows)
out_metrics = os.path.join(metrics_dir, f"spatialCV_metrics_{start_idx:03d}_{end_idx:03d}.csv")
metrics_df.to_csv(out_metrics, index=False)

summary = (
    metrics_df.groupby(["decluster","model"])
    .agg(R2_mean=("R2","mean"), RMSE_mean=("RMSE","mean"), MAE_mean=("MAE","mean"))
    .reset_index()
)
out_summary = os.path.join(metrics_dir, f"spatialCV_summary_{start_idx:03d}_{end_idx:03d}.csv")
summary.to_csv(out_summary, index=False)

print(f"\n✅ Spatial CV done for {len(subset)} declusters.")
print(f"📁 Metrics saved to {out_metrics}")
print(f"📁 Summary saved to {out_summary}")



🚀 Running spatial CV for declusters 1–10

🧩 Decluster 001

🧩 Decluster 002

🧩 Decluster 003

🧩 Decluster 004

🧩 Decluster 005

🧩 Decluster 006

🧩 Decluster 007

🧩 Decluster 008

🧩 Decluster 009

🧩 Decluster 010

✅ Spatial CV done for 10 declusters.
📁 Metrics saved to /Users/inesschwartz/Desktop/model/results_spatialCV/spatialCV_metrics_001_010.csv
📁 Summary saved to /Users/inesschwartz/Desktop/model/results_spatialCV/spatialCV_summary_001_010.csv


In [2]:
# =========================================================
# SOC_predict_national_small_batches.py
# =========================================================
# Run RF, OK, RF+OK national predictions in very small batches
# for stability on limited memory systems.
# =========================================================

import os, glob, gc
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import rowcol
from sklearn.ensemble import RandomForestRegressor
from pykrige.ok import OrdinaryKriging

# ---------- Paths ----------
decluster_dir = "/Users/inesschwartz/Desktop/model/decluster_runs_aligned"
pred_grid_csv = "/Users/inesschwartz/Desktop/model/covariates_stack_1km_utm33s_fixed.csv"
ref_raster    = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/slope_height.tif"
out_dir       = "/Users/inesschwartz/Desktop/model/ensemble_preds"
os.makedirs(out_dir, exist_ok=True)

# ---------- Model settings ----------
target = "log_soc_stock"
features = [
    "MRRTF", "MRVBF", "annual_precip", "grazing_1950", "cropland_1950",
    "precip_wettest_month", "relief_TRI", "standardized_height",
    "temp_annual_range", "terrain_surf_convexity", "terrain_surf_texture",
    "tmax_mean", "valley_depth", "faosoil_id", "slope_height"
]
rf_params = dict(n_estimators=500, min_samples_leaf=3, max_features=0.5,
                 max_depth=20, random_state=42, n_jobs=2)  # use fewer jobs to reduce memory
vmodel = "gaussian"
vparams = {"nugget": 0.0, "sill": 0.18, "range": 14000}

# ---------- Load prediction grid ----------
df_pred = pd.read_csv(pred_grid_csv)
X_pred = df_pred[features].astype("float32").copy()
coords_pred = df_pred[["X_coord", "Y_coord"]].astype("float32").values

with rasterio.open(ref_raster) as ref:
    profile = ref.profile
    transform = ref.transform
    width, height = ref.width, ref.height

rows, cols = rowcol(transform, df_pred["X_coord"], df_pred["Y_coord"])
rows, cols = np.array(rows), np.array(cols)
valid_mask = ((rows >= 0) & (rows < height) & (cols >= 0) & (cols < width))

# ---------- Helper functions ----------
def krige_predict(xy_train, z_train, xy_test):
    ok = OrdinaryKriging(
        x=xy_train[:,0], y=xy_train[:,1], z=z_train,
        variogram_model=vmodel, variogram_parameters=vparams,
        enable_plotting=False, verbose=False
    )
    pred, _ = ok.execute("points", xy_test[:,0], xy_test[:,1])
    return np.asarray(pred)

def save_raster(values_bt, out_path):
    arr = np.full((height, width), np.nan, dtype="float32")
    arr[rows[valid_mask], cols[valid_mask]] = values_bt[valid_mask].astype("float32")
    pf = profile.copy()
    pf.update(dtype="float32", count=1, compress="lzw", nodata=np.nan)
    with rasterio.open(out_path, "w", **pf) as dst:
        dst.write(arr, 1)

# ---------- Define decluster batches ----------
decluster_files = sorted(glob.glob(os.path.join(decluster_dir, "decluster_run_*.csv")))

# 💡 Adjust this for how many to run per batch
batch_ranges = [(1,5)]  # run 1–5 declusters at a time
# batch_ranges = [(6,10)]
# batch_ranges = [(11,15)], etc.

# ---------- Run loop ----------
for (start_idx, end_idx) in batch_ranges:
    print(f"\n🚀 Running declusters {start_idx}–{end_idx}")
    subset = decluster_files[start_idx - 1:end_idx]

    for path in subset:
        it = int(os.path.basename(path).split("_")[-1].split(".")[0])
        print(f"\n🧩 Decluster {it:03d}")

        # Skip already done
        all_exist = all(
            os.path.exists(os.path.join(out_dir, f"{mdl}_decluster_{it:03d}.tif"))
            for mdl in ["RF", "OK", "RF_OK"]
        )
        if all_exist:
            print(f"⚠️ Skipping decluster {it:03d} (rasters exist).")
            continue

        df = pd.read_csv(path)
        df = df.dropna(subset=[target] + features + ["X_coord", "Y_coord"])

        X = df[features].astype("float32").values
        y = df[target].astype("float32").values
        XY = df[["X_coord", "Y_coord"]].astype("float32").values

        # ---- RF ----
        rf = RandomForestRegressor(**rf_params).fit(X, y)
        pred_rf = rf.predict(X_pred)
        del rf
        gc.collect()

        # ---- OK ----
        pred_ok = krige_predict(XY, y, coords_pred)

        # ---- RF+OK ----
        rf_tmp = RandomForestRegressor(**rf_params).fit(X, y)
        resid = y - rf_tmp.predict(X)
        del rf_tmp
        gc.collect()

        pred_resid = krige_predict(XY, resid, coords_pred)
        pred_rfok = pred_rf + pred_resid
        del pred_resid
        gc.collect()

        # ---- Save ----
        preds_bt = {
            "RF": np.expm1(pred_rf),
            "OK": np.expm1(pred_ok),
            "RF_OK": np.expm1(pred_rfok)
        }
        for mdl, vals in preds_bt.items():
            out_tif = os.path.join(out_dir, f"{mdl}_decluster_{it:03d}.tif")
            save_raster(vals, out_tif)
            print(f"💾 Saved {mdl}_decluster_{it:03d}.tif")

        del pred_rf, pred_ok, pred_rfok, preds_bt, X, y, XY
        gc.collect()

    print(f"\n✅ Finished declusters {start_idx}–{end_idx}\n{'='*60}")



🚀 Running declusters 1–5

🧩 Decluster 001


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
## make the final ensemble rasters (once both metrics and per cluster predictions are completed)
## run inverse variance aggregation 

In [8]:
# =========================================================
# ENSEMBLE AGGREGATION SCRIPT — Mean & Std Rasters (with back-transform)
# =========================================================
# Purpose:
#   Aggregate 50 decluster prediction rasters (RF, OK, RF_OK)
#   into ensemble mean and std rasters (in SOC stock units).
#
# Expected input:
#   /Users/inesschwartz/Desktop/model/ensemble_preds/
#   containing e.g.:
#       RF_decluster_001.tif ... RF_decluster_050.tif
#       OK_decluster_001.tif ... OK_decluster_050.tif
#       RF_OK_decluster_001.tif ... RF_OK_decluster_050.tif
#
# Output:
#   ensemble_mean_RF.tif
#   ensemble_std_RF.tif
#   ensemble_mean_OK.tif
#   ensemble_std_OK.tif
#   ensemble_mean_RF_OK.tif
#   ensemble_std_RF_OK.tif
# =========================================================

import os
import glob
import numpy as np
import rasterio

# =========================================================
# PATHS
# =========================================================
input_dir = "/Users/inesschwartz/Desktop/model/ensemble_preds"
output_dir = input_dir  # save to same folder

# =========================================================
# MODELS TO AGGREGATE
# =========================================================
models = ["RF", "OK", "RF_OK"]

# =========================================================
# FUNCTION TO AGGREGATE ENSEMBLE
# =========================================================
def aggregate_ensemble(model_name):
    print(f"\n📊 Aggregating ensemble for model: {model_name}")

    # List of rasters for this model
    raster_files = sorted(glob.glob(os.path.join(input_dir, f"{model_name}_decluster_*.tif")))
    if len(raster_files) == 0:
        print(f"⚠️ No rasters found for {model_name}. Skipping.")
        return

    # Read first raster to get metadata
    with rasterio.open(raster_files[0]) as src:
        profile = src.profile
        height, width = src.height, src.width
        crs = src.crs
        transform = src.transform

    # Initialize accumulators
    sum_arr = np.zeros((height, width), dtype="float64")
    sumsq_arr = np.zeros((height, width), dtype="float64")
    count_arr = np.zeros((height, width), dtype="int32")

    # Loop through decluster rasters
    for idx, file in enumerate(raster_files, start=1):
        with rasterio.open(file) as src:
            data = src.read(1).astype("float64")

            # Skip all-NaN or empty rasters
            if np.all(np.isnan(data)):
                print(f"   ⚠️ Skipping empty raster: {file}")
                continue

            # --- Back-transform from log-space to SOC (Mg C ha⁻¹) ---
            data_bt = np.expm1(data)
            mask = ~np.isnan(data_bt)

            sum_arr[mask] += data_bt[mask]
            sumsq_arr[mask] += data_bt[mask] ** 2
            count_arr[mask] += 1

        print(f"   ✔️ Added {model_name}_decluster_{idx:03d}.tif")

    # Avoid division by zero
    valid_mask = count_arr > 0

    # Compute mean and standard deviation
    mean_arr = np.full((height, width), np.nan, dtype="float64")
    std_arr = np.full((height, width), np.nan, dtype="float64")

    mean_arr[valid_mask] = sum_arr[valid_mask] / count_arr[valid_mask]
    var_arr = (sumsq_arr[valid_mask] / count_arr[valid_mask]) - mean_arr[valid_mask] ** 2
    std_arr[valid_mask] = np.sqrt(np.maximum(var_arr, 0))  # handle rounding negatives

    # Update profile for float32 output
    profile.update(dtype="float32", count=1, compress="lzw", nodata=np.nan)

    # Write mean raster
    out_mean = os.path.join(output_dir, f"ensemble_mean_{model_name}.tif")
    with rasterio.open(out_mean, "w", **profile) as dst:
        dst.write(mean_arr.astype("float32"), 1)
    print(f"💾 Saved mean raster: {out_mean}")

    # Write std raster
    out_std = os.path.join(output_dir, f"ensemble_std_{model_name}.tif")
    with rasterio.open(out_std, "w", **profile) as dst:
        dst.write(std_arr.astype("float32"), 1)
    print(f"💾 Saved std raster: {out_std}")

# =========================================================
# RUN FOR ALL MODELS
# =========================================================
for model in models:
    aggregate_ensemble(model)

print("\n🎯 Ensemble aggregation complete for all models (in SOC stock units).")



📊 Aggregating ensemble for model: RF
   ✔️ Added RF_decluster_001.tif
   ✔️ Added RF_decluster_002.tif
   ✔️ Added RF_decluster_003.tif
   ✔️ Added RF_decluster_004.tif
   ✔️ Added RF_decluster_005.tif
   ✔️ Added RF_decluster_006.tif
   ✔️ Added RF_decluster_007.tif
   ✔️ Added RF_decluster_008.tif
   ✔️ Added RF_decluster_009.tif
   ✔️ Added RF_decluster_010.tif
💾 Saved mean raster: /Users/inesschwartz/Desktop/model/ensemble_preds/ensemble_mean_RF.tif
💾 Saved std raster: /Users/inesschwartz/Desktop/model/ensemble_preds/ensemble_std_RF.tif

📊 Aggregating ensemble for model: OK
   ✔️ Added OK_decluster_001.tif
   ✔️ Added OK_decluster_002.tif
   ✔️ Added OK_decluster_003.tif
   ✔️ Added OK_decluster_004.tif
   ✔️ Added OK_decluster_005.tif
   ✔️ Added OK_decluster_006.tif
   ✔️ Added OK_decluster_007.tif
   ✔️ Added OK_decluster_008.tif
   ✔️ Added OK_decluster_009.tif
   ✔️ Added OK_decluster_010.tif


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_1499/745305238.py:101: RuntimeWarning: overflow encountered in cast
  dst.write(mean_arr.astype("float32"), 1)
/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_1499/745305238.py:107: RuntimeWarning: overflow encountered in cast
  dst.write(std_arr.astype("float32"), 1)


💾 Saved mean raster: /Users/inesschwartz/Desktop/model/ensemble_preds/ensemble_mean_OK.tif
💾 Saved std raster: /Users/inesschwartz/Desktop/model/ensemble_preds/ensemble_std_OK.tif

📊 Aggregating ensemble for model: RF_OK
   ✔️ Added RF_OK_decluster_001.tif
   ✔️ Added RF_OK_decluster_002.tif
   ✔️ Added RF_OK_decluster_003.tif
   ✔️ Added RF_OK_decluster_004.tif
   ✔️ Added RF_OK_decluster_005.tif
   ✔️ Added RF_OK_decluster_006.tif
   ✔️ Added RF_OK_decluster_007.tif
   ✔️ Added RF_OK_decluster_008.tif
   ✔️ Added RF_OK_decluster_009.tif
   ✔️ Added RF_OK_decluster_010.tif
💾 Saved mean raster: /Users/inesschwartz/Desktop/model/ensemble_preds/ensemble_mean_RF_OK.tif
💾 Saved std raster: /Users/inesschwartz/Desktop/model/ensemble_preds/ensemble_std_RF_OK.tif

🎯 Ensemble aggregation complete for all models (in SOC stock units).


In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/inesschwartz/Desktop/model/results_final/rf_model_performance_tuned.csv")
print("Mean R²:", df["R2"].mean())
print("Mean RMSE:", df["RMSE"].mean())
print("Mean MAE:", df["MAE"].mean())
print("Std R²:", df["R2"].std())


Mean R²: 0.32153962074743575
Mean RMSE: 0.4383426137430786
Mean MAE: 0.3219464170521836
Std R²: 0.011843891828476399


## something didn't work above. need to check grid alignment!

In [2]:
# =========================================================
# FINAL SOC MAP — Random Forest (full model)
# =========================================================

import pandas as pd
import numpy as np
import rasterio
from rasterio.transform import from_origin
from sklearn.ensemble import RandomForestRegressor
import os

# =========================================================
# PATHS
# =========================================================
train_csv = "/Users/inesschwartz/Desktop/model/train_final.csv"  # Final training dataset (15 covariates)
pred_grid_csv = "/Users/inesschwartz/Desktop/model/covariates_stack_1km_utm33s.csv"  # Aligned prediction grid
output_raster = "/Users/inesschwartz/Desktop/model/SOC_RF_final.tif"

# =========================================================
# RANDOM FOREST PARAMETERS (from tuning)
# =========================================================
rf_params = {
    "n_estimators": 1500,
    "min_samples_leaf": 3,
    "max_features": 0.5,
    "max_depth": None,
    "random_state": 42,
    "n_jobs": -1
}

# =========================================================
# 1️⃣ LOAD DATA
# =========================================================
df_train = pd.read_csv(train_csv)
df_pred = pd.read_csv(pred_grid_csv)

target = "log_soc_stock"

# Define same feature order as used during training
features = [
    "MRRTF", "MRVBF", "annual_precip", "grazing_1950", "cropland_1950",
    "precip_wettest_month", "relief_TRI", "standardized_height", "temp_annual_range",
    "terrain_surf_convexity", "terrain_surf_texture", "tmax_mean", "valley_depth",
    "faosoil_id", "slope_height"
]

# Make sure both datasets have same columns and order
X_train = df_train[features].copy()
y_train = df_train[target].copy()
X_pred = df_pred[features].copy()

print(f"✅ Training samples: {len(df_train)}")
print(f"✅ Prediction grid points: {len(df_pred)}")

# =========================================================
# 2️⃣ TRAIN FINAL RF MODEL
# =========================================================
rf = RandomForestRegressor(**rf_params)
rf.fit(X_train, y_train)

print("🎯 Final Random Forest model trained on full dataset.")

# =========================================================
# 3️⃣ PREDICT OVER 1 KM GRID
# =========================================================
df_pred["pred_log_soc"] = rf.predict(X_pred)
df_pred["pred_soc"] = np.exp(df_pred["pred_log_soc"])  # back-transform log SOC if applicable

print("📈 Predictions completed — generating raster...")

# =========================================================
# 4️⃣ CREATE OUTPUT RASTER
# =========================================================
# Load any aligned reference raster for transform and shape
ref_raster = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/slope_height.tif"

with rasterio.open(ref_raster) as ref:
    profile = ref.profile
    transform = ref.transform
    width = ref.width
    height = ref.height
    crs = ref.crs

# Create an empty raster grid and fill values
soc_pred_raster = np.full((height, width), np.nan, dtype="float32")

# Convert coordinates to row/col
rows, cols = rasterio.transform.rowcol(transform, df_pred["X_coord"], df_pred["Y_coord"])
valid_mask = (np.array(rows) >= 0) & (np.array(rows) < height) & (np.array(cols) >= 0) & (np.array(cols) < width)

soc_pred_raster[rows[valid_mask], cols[valid_mask]] = df_pred.loc[valid_mask, "pred_soc"]

# Update raster metadata
profile.update(dtype="float32", count=1, compress="lzw", nodata=np.nan)

# Write raster
with rasterio.open(output_raster, "w", **profile) as dst:
    dst.write(soc_pred_raster, 1)

print(f"✅ Final SOC stock map saved to:\n{output_raster}")


✅ Training samples: 725
✅ Prediction grid points: 1252955
🎯 Final Random Forest model trained on full dataset.
📈 Predictions completed — generating raster...
✅ Final SOC stock map saved to:
/Users/inesschwartz/Desktop/model/SOC_RF_final.tif


In [4]:
# =========================================================
# SOC Ensemble Uncertainty Map (RF and RF+OK)
# =========================================================

import os
import glob
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import rowcol

# --- Paths ---
pred_dir = "/Users/inesschwartz/Desktop/model/predictions"
ref_raster = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/slope_height.tif"
out_dir = "/Users/inesschwartz/Desktop/model/ensemble_maps"
os.makedirs(out_dir, exist_ok=True)

# --- Load reference raster for spatial alignment ---
with rasterio.open(ref_raster) as ref:
    profile = ref.profile
    transform = ref.transform
    width, height = ref.width, ref.height
    crs = ref.crs

print(f"📏 Reference grid: {width} × {height} | CRS: {crs}")

# --- Collect all prediction CSVs ---
csv_files = sorted(glob.glob(os.path.join(pred_dir, "predictions_iter_*.csv")))
print(f"📂 Found {len(csv_files)} prediction files.")

# --- Containers ---
rf_preds, rfok_preds = [], []
coords_ref = None

# --- Read each decluster prediction ---
for i, f in enumerate(csv_files, start=1):
    df = pd.read_csv(f)
    if coords_ref is None:
        coords_ref = df[['X_coord', 'Y_coord']].copy()
    rf_preds.append(df['pred_RF'].values)
    rfok_preds.append(df['pred_RF_OK'].values)
    print(f"✅ Loaded iteration {i:02d}: {f}")

# --- Stack predictions ---
rf_stack = np.column_stack(rf_preds)
rfok_stack = np.column_stack(rfok_preds)

# --- Compute ensemble statistics ---
rf_mean = np.nanmean(rf_stack, axis=1)
rf_std = np.nanstd(rf_stack, axis=1)
rfok_mean = np.nanmean(rfok_stack, axis=1)
rfok_std = np.nanstd(rfok_stack, axis=1)

# --- Create raster arrays ---
rf_mean_raster = np.full((height, width), np.nan, dtype="float32")
rf_std_raster = np.full((height, width), np.nan, dtype="float32")
rfok_mean_raster = np.full((height, width), np.nan, dtype="float32")
rfok_std_raster = np.full((height, width), np.nan, dtype="float32")

# --- Convert coordinates to row/col positions ---
rows, cols = rasterio.transform.rowcol(transform, coords_ref["X_coord"], coords_ref["Y_coord"])
valid_mask = (
    (np.array(rows) >= 0)
    & (np.array(rows) < height)
    & (np.array(cols) >= 0)
    & (np.array(cols) < width)
)

rf_mean_raster[rows[valid_mask], cols[valid_mask]] = rf_mean[valid_mask]
rf_std_raster[rows[valid_mask], cols[valid_mask]] = rf_std[valid_mask]
rfok_mean_raster[rows[valid_mask], cols[valid_mask]] = rfok_mean[valid_mask]
rfok_std_raster[rows[valid_mask], cols[valid_mask]] = rfok_std[valid_mask]

# --- Update profile ---
profile.update(dtype="float32", count=1, compress="lzw", nodata=np.nan)

# --- Save rasters ---
out_paths = {
    "SOC_RF_mean": rf_mean_raster,
    "SOC_RF_std": rf_std_raster,
    "SOC_RFOK_mean": rfok_mean_raster,
    "SOC_RFOK_std": rfok_std_raster,
}

for name, arr in out_paths.items():
    out_path = os.path.join(out_dir, f"{name}.tif")
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(arr, 1)
    print(f"✅ Saved {name} raster → {out_path}")

print("\n🎉 Ensemble mean and uncertainty rasters created successfully!")


📏 Reference grid: 1379 × 1532 | CRS: EPSG:32733
📂 Found 50 prediction files.
✅ Loaded iteration 01: /Users/inesschwartz/Desktop/model/predictions/predictions_iter_001.csv
✅ Loaded iteration 02: /Users/inesschwartz/Desktop/model/predictions/predictions_iter_002.csv
✅ Loaded iteration 03: /Users/inesschwartz/Desktop/model/predictions/predictions_iter_003.csv
✅ Loaded iteration 04: /Users/inesschwartz/Desktop/model/predictions/predictions_iter_004.csv
✅ Loaded iteration 05: /Users/inesschwartz/Desktop/model/predictions/predictions_iter_005.csv
✅ Loaded iteration 06: /Users/inesschwartz/Desktop/model/predictions/predictions_iter_006.csv
✅ Loaded iteration 07: /Users/inesschwartz/Desktop/model/predictions/predictions_iter_007.csv
✅ Loaded iteration 08: /Users/inesschwartz/Desktop/model/predictions/predictions_iter_008.csv
✅ Loaded iteration 09: /Users/inesschwartz/Desktop/model/predictions/predictions_iter_009.csv
✅ Loaded iteration 10: /Users/inesschwartz/Desktop/model/predictions/predicti

## REsuls still very poor...checking 1 decluster at a time

In [1]:
# =========================================================
# SOC_RF_declu001.py
# =========================================================
# Random Forest: 5-fold spatial CV + full prediction
# =========================================================

import os, gc
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import rowcol
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# ---------- Paths ----------
decluster_csv = "/Users/inesschwartz/Desktop/model/decluster_runs_aligned/decluster_run_001.csv"
pred_grid_csv = "/Users/inesschwartz/Desktop/model/covariates_stack_1km_utm33s_fixed.csv"
ref_raster    = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/slope_height.tif"
out_dir       = "/Users/inesschwartz/Desktop/model/ensemble_preds"
metrics_dir   = "/Users/inesschwartz/Desktop/model/results_spatialCV"
os.makedirs(out_dir, exist_ok=True)
os.makedirs(metrics_dir, exist_ok=True)

target = "log_soc_stock"
features = [
    "MRRTF", "MRVBF", "annual_precip", "grazing_1950", "cropland_1950",
    "precip_wettest_month", "relief_TRI", "standardized_height",
    "temp_annual_range", "terrain_surf_convexity", "terrain_surf_texture",
    "tmax_mean", "valley_depth", "faosoil_id", "slope_height"
]
rf_params = dict(n_estimators=1000, min_samples_leaf=3, max_features=None,
                 max_depth=30, random_state=42, n_jobs=4)

# ---------- Load data ----------
df = pd.read_csv(decluster_csv)
df = df.dropna(subset=[target]+features+["X_coord","Y_coord"])

X = df[features].astype("float32").values
y = df[target].astype("float32").values
XY = df[["X_coord","Y_coord"]].astype("float32").values

# ---------- Spatial CV ----------
def block_groups(df, block_m=10000):
    gx = (df["X_coord"] // block_m).astype(int)
    gy = (df["Y_coord"] // block_m).astype(int)
    return (gx.astype(str) + "_" + gy.astype(str)).values

groups = block_groups(df)
cv = GroupKFold(n_splits=5)

metrics = []
for fold, (tr, te) in enumerate(cv.split(X, y, groups=groups), 1):
    rf = RandomForestRegressor(**rf_params).fit(X[tr], y[tr])
    y_pred = rf.predict(X[te])
    metrics.append(dict(
        fold=fold,
        R2=r2_score(y[te], y_pred),
        RMSE=np.sqrt(mean_squared_error(y[te], y_pred)),
        MAE=np.mean(np.abs(y[te]-y_pred))
    ))
pd.DataFrame(metrics).to_csv(f"{metrics_dir}/spatialCV_RF_001.csv", index=False)

# ---------- Full training + prediction ----------
rf_full = RandomForestRegressor(**rf_params).fit(X, y)

df_pred = pd.read_csv(pred_grid_csv)
X_pred = df_pred[features].astype("float32").copy()
coords_pred = df_pred[["X_coord","Y_coord"]].astype("float32").values

with rasterio.open(ref_raster) as ref:
    profile = ref.profile
    transform = ref.transform
    width, height = ref.width, ref.height
rows, cols = rowcol(transform, df_pred["X_coord"], df_pred["Y_coord"])
rows, cols = np.array(rows), np.array(cols)
valid_mask = ((rows>=0)&(rows<height)&(cols>=0)&(cols<width))

pred_rf = np.expm1(rf_full.predict(X_pred))  # back-transform
arr = np.full((height, width), np.nan, dtype="float32")
arr[rows[valid_mask], cols[valid_mask]] = pred_rf[valid_mask].astype("float32")
profile.update(dtype="float32", count=1, compress="lzw", nodata=np.nan)
out_tif = os.path.join(out_dir, "RF_decluster_001.tif")
with rasterio.open(out_tif, "w", **profile) as dst:
    dst.write(arr, 1)
print(f"💾 Saved RF raster → {out_tif}")


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


💾 Saved RF raster → /Users/inesschwartz/Desktop/model/ensemble_preds/RF_decluster_001.tif


In [3]:
import geopandas as gpd
from shapely.geometry import Point
from sklearn.model_selection import GroupKFold

# Load the same decluster dataset
df = pd.read_csv("/Users/inesschwartz/Desktop/model/decluster_runs_aligned/decluster_run_001.csv")

# Same grouping rule as in your model
def block_groups(df, block_m=10000):
    gx = (df["X_coord"] // block_m).astype(int)
    gy = (df["Y_coord"] // block_m).astype(int)
    return (gx.astype(str) + "_" + gy.astype(str)).values

groups = block_groups(df)
cv = GroupKFold(n_splits=5)

# Get indices for each fold
folds = list(cv.split(df, df["log_soc_stock"], groups=groups))

# Get fold 1 test indices (the evaluation subset)
train_idx, test_idx = folds[0]
fold1 = df.iloc[test_idx].copy()

# Convert to GeoDataFrame for visualization
gdf_fold1 = gpd.GeoDataFrame(
    fold1,
    geometry=gpd.points_from_xy(fold1.X_coord, fold1.Y_coord),
    crs="EPSG:32733"   # UTM 33S (Angola)
)

gdf_fold1.to_file("/Users/inesschwartz/Desktop/model/results_spatialCV/fold1_points.gpkg", driver="GPKG")
print(f"✅ Saved fold 1 points → fold1_points.gpkg  (n={len(gdf_fold1)})")


✅ Saved fold 1 points → fold1_points.gpkg  (n=126)


In [5]:
print("Fold 1 sample count:", len(fold1))
print("Mean SOC (log):", fold1[target].mean())
print("SOC variance:", fold1[target].var())


Fold 1 sample count: 126
Mean SOC (log): 1.1582292436903145
SOC variance: 0.25384484504037236


In [1]:
# =========================================================
# SOC_OK_declu001.py
# =========================================================
# Ordinary Kriging: spatial CV + full prediction (blocked)
# =========================================================

import os, gc
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import rowcol
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pykrige.ok import OrdinaryKriging

decluster_csv = "/Users/inesschwartz/Desktop/model/decluster_runs_aligned/decluster_run_001.csv"
pred_grid_csv = "/Users/inesschwartz/Desktop/model/covariates_stack_1km_utm33s_fixed.csv"
ref_raster    = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/slope_height.tif"
out_dir       = "/Users/inesschwartz/Desktop/model/ensemble_preds"
metrics_dir   = "/Users/inesschwartz/Desktop/model/results_spatialCV"
os.makedirs(out_dir, exist_ok=True)
os.makedirs(metrics_dir, exist_ok=True)

target = "log_soc_stock"
variogram_model = "gaussian"
variogram_params = {"nugget": 0.000, "sill": 0.181, "range": 14206}
BLOCK_SIZE = 10000

# ---------- Load data ----------
df = pd.read_csv(decluster_csv).dropna(subset=[target,"X_coord","Y_coord"])
y = df[target].astype("float32").values
XY = df[["X_coord","Y_coord"]].astype("float32").values

# ---------- Spatial CV ----------
def block_groups(df, block_m=10000):
    gx = (df["X_coord"] // block_m).astype(int)
    gy = (df["Y_coord"] // block_m).astype(int)
    return (gx.astype(str) + "_" + gy.astype(str)).values

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def krige_predict(xy_train, z_train, xy_test):
    ok = OrdinaryKriging(
        x=xy_train[:,0], y=xy_train[:,1], z=z_train,
        variogram_model=variogram_model,
        variogram_parameters=variogram_params,
        enable_plotting=False, verbose=False
    )
    pred, _ = ok.execute("points", xy_test[:,0], xy_test[:,1])
    return np.asarray(pred)

groups = block_groups(df)
cv = GroupKFold(n_splits=5)
metrics = []
for fold, (tr, te) in enumerate(cv.split(XY, y, groups=groups), 1):
    preds = krige_predict(XY[tr], y[tr], XY[te])
    metrics.append(dict(fold=fold, R2=r2_score(y[te], preds),
                        RMSE=rmse(y[te], preds), MAE=np.mean(np.abs(y[te]-preds))))
pd.DataFrame(metrics).to_csv(f"{metrics_dir}/spatialCV_OK_001.csv", index=False)

# ---------- National prediction (blocked) ----------
df_pred = pd.read_csv(pred_grid_csv)
coords_pred = df_pred[["X_coord","Y_coord"]].astype("float32").values
with rasterio.open(ref_raster) as ref:
    profile = ref.profile
    transform = ref.transform
    width, height = ref.width, ref.height
rows, cols = rowcol(transform, df_pred["X_coord"], df_pred["Y_coord"])
rows, cols = np.array(rows), np.array(cols)
valid_mask = ((rows>=0)&(rows<height)&(cols>=0)&(cols<width))

ok_full = OrdinaryKriging(
    x=XY[:,0], y=XY[:,1], z=y,
    variogram_model=variogram_model,
    variogram_parameters=variogram_params,
    enable_plotting=False, verbose=False
)
preds = []
for i in range(0, len(coords_pred), BLOCK_SIZE):
    sub = coords_pred[i:i+BLOCK_SIZE]
    pred_block, _ = ok_full.execute("points", sub[:,0], sub[:,1])
    preds.append(pred_block)
pred_ok = np.concatenate(preds)

# Back-transform and save
soc_ok = np.expm1(pred_ok)
arr = np.full((height, width), np.nan, dtype="float32")
arr[rows[valid_mask], cols[valid_mask]] = soc_ok[valid_mask].astype("float32")
profile.update(dtype="float32", count=1, compress="lzw", nodata=np.nan)
out_tif = os.path.join(out_dir, "OK_decluster_001.tif")
with rasterio.open(out_tif, "w", **profile) as dst:
    dst.write(arr, 1)
print(f"💾 Saved OK raster → {out_tif}")


💾 Saved OK raster → /Users/inesschwartz/Desktop/model/ensemble_preds/OK_decluster_001.tif


In [6]:
ok_pred_log = pred_ok  # before np.expm1()
print("OK log-space min/max:", np.nanmin(ok_pred_log), np.nanmax(ok_pred_log))


OK log-space min/max: -0.10891051699397367 3.5822206650881334


In [8]:
print("Training SOC (Mg C ha⁻¹) min/max:",
      np.expm1(y.min()), np.expm1(y.max()))


Training SOC (Mg C ha⁻¹) min/max: 0.0 35.253143
